# Description

Script to run the agent on the hardware

0. Description
1. Imports
2. Paths
3. Configuration
4. Functions
5. Creation
6. Run on Hardware
7. Save recorded Data


# Imports

In [1]:
# General imports
import os
import datetime
from os.path import expanduser
import pandas
import pickle
import numpy
from matplotlib import pyplot as plt

# rllib imports
import ray
import ray.rllib.agents.ppo as ppo
from ray.tune.registry import get_trainable_cls
from ray.tune.registry import register_env
from ray.rllib.env import MultiAgentEnv
from ray.rllib.env.base_env import _DUMMY_AGENT_ID
from ray.rllib.evaluation.worker_set import WorkerSet
from ray.rllib.policy.sample_batch import DEFAULT_POLICY_ID
from ray.rllib.utils.deprecation import deprecation_warning

# Gym imports
from gym.envs.robotics.ur10 import randomize
from gym.envs.robotics import ur10_noisy_pd_env
from gym import utils
from gym.envs.robotics import rotations, robot_custom_env, utils

# MuJoCo imports
import mujoco_py

# ROS imports
import rospy
from sensor_msgs.msg import JointState
from std_msgs.msg import Float64MultiArray
from std_msgs.msg import String
from geometry_msgs.msg import WrenchStamped
from realsense2_camera.msg import CustomVector
from realsense2_camera.msg import EstimatedPose

/home/lars/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lars/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lars/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lars/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

# Paths

In [2]:
HOME_PATH = os.getenv("HOME")
PROJECT_PATH = os.path.join(*[HOME_PATH, "DRL_AI4RoMoCo"])
SAVE_PATH = os.path.join(*[PROJECT_PATH, "code", "data"])
XML_MODEL_PATH = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation","UR10","ur10_heg.xml"])

# Configuration

Hardware - Environment (ROS)

In [ ]:
actual_q = numpy.zeros(6,)
ft_values = numpy.zeros(6,)

Simulation - Environment (MuJoCo, Gym)

In [ ]:
goal = numpy.array([ 0.69423743, -0.83110109,  1.17388998, -1.57161506,  0.02185773, -3.14102438])
print(goal)

In [ ]:
x_pos = sim.data.get_body_xpos("gripper_dummy_heg")
x_mat = sim.data.get_body_xmat("gripper_dummy_heg")
rpy =  normalize_rad(rotations.mat2euler(x_mat))
print(x_pos, rpy)

In [ ]:
initial_qpos_simpheg_conf2 = numpy.array([0, -1.3, 2.1, -0.80, 1.5708, 0])
randomize_kwargs = {
    'var_mass':     0,#0.5,
    'var_damp':     0,#0.5,
    'var_fr':       0,#0.5,
    'var_grav_x_y': 0,#0.5,
    'var_grav_z':   0,#1,
    'var_body_pos': 0,#0.05,
    'var_body_rot': 0,
}

Agent (ray, tune, rllib)

In [ ]:
checkpoint = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation", "final_models",
                            "model1", "checkpoint-2120"])

In [ ]:
checkpoint = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation", "final_models",
                            "model2", "checkpoint-3440"])

In [ ]:
checkpoint = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation", "final_models",
                            "model3", "checkpoint-3030"])

In [ ]:
checkpoint = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation", "final_models",
                            "model4", "checkpoint-3590"])

In [ ]:
checkpoint = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation", "final_models",
                            "model5", "checkpoint-4410"])

In [ ]:
checkpoint

In [ ]:
config = {}
    # Load configuration from checkpoint file.
config_dir = os.path.dirname(checkpoint)
config_path = os.path.join(config_dir, "params.pkl")
# Try parent directory.
if not os.path.exists(config_path):
    config_path = os.path.join(config_dir, "../params.pkl")
if os.path.exists(config_path):
    print("Config found!")
    with open(config_path, "rb") as f:
            config = pickle.load(f)

In [ ]:
config['num_workers'] = 2
config['num_gpus'] = 1

In [ ]:
config

# Functions

Helper Functions

In [ ]:
def normalize_rad(angles):
    angles = numpy.array(angles)
    angles = angles % (2*numpy.pi)
    angles = (angles + 2*numpy.pi) % (2*numpy.pi)
    for i in range(len(angles)):
        if (angles[i] > numpy.pi):
            angles[i] -= 2*numpy.pi
    return angles

def get_obs():
    set_state(actual_q)
    for i in range(10):
        viewer.render()
    #print(ft_values)
    x_pos = sim.data.get_body_xpos("gripper_dummy_heg")
    x_mat = sim.data.get_body_xmat("gripper_dummy_heg")
    rpy =  normalize_rad(rotations.mat2euler(x_mat))
    #print(x_pos, rpy)

    obs = numpy.concatenate([
                x_mat.dot(x_pos-goal[:3]), x_mat.dot(normalize_rad(rpy-goal[3:])), ft_values.copy()
            ])
    return obs

def get_dq(dx):
    jacp = sim.data.get_body_jacp(name="gripper_dummy_heg").reshape(3, 6)
    jacr = sim.data.get_body_jacr(name="gripper_dummy_heg").reshape(3, 6)
    jac = numpy.vstack((jacp, jacr))
    dq = numpy.linalg.lstsq(jac, dx)[0].reshape(6, )
    return dq

def go_smooth(q):
    rate = rospy.Rate(125)
    dq = q - actual_q
    while numpy.linalg.norm(dq)>0.005:
        dq = q - actual_q
        dq_ = numpy.clip(dq, -0.0001, 0.0001)
        control(ctrl_q+dq_)
        rate.sleep()
    control(q)
    
def set_state(qpos):
    assert qpos.shape == (model.nq,)
    old_state = sim.get_state()
    new_state = mujoco_py.MjSimState(old_state.time, qpos, old_state.qvel,
                                     old_state.act, old_state.udd_state)
    sim.set_state(new_state)
    sim.forward()
    
def keep_going(steps, num_steps, episodes, num_episodes):
    """Determine whether we've collected enough data"""
    # if num_episodes is set, this overrides num_steps
    if num_episodes:
        return episodes < num_episodes
    # if num_steps is set, continue until we reach the limit
    if num_steps:
        return steps < num_steps
    # otherwise keep going forever
    return True

ROS

In [1]:
# ROS callback functions
def q_callback(data):
    '''Callback for the joint angles, velocities and accelerations'''
    global actual_q
    global actual_qd
    global actual_qdd
    actual_q = numpy.array(data.position)[[2,1,0,3,4,5]]
    actual_qd = numpy.array(data.velocity)[[2,1,0,3,4,5]]
    actual_qdd = numpy.array(data.effort)[[2,1,0,3,4,5]]
    set_state(actual_q)
    viewer.render()
    
def ft_callback(data):
    '''Callback for the Robotiq FT300 force torque values'''
    global ft_values
    ft_values = - 5 * numpy.array([data.wrench.force.x, data.wrench.force.y, data.wrench.force.z, data.wrench.torque.x, \
                            data.wrench.torque.y, data.wrench.torque.z])
    
def pose_callback(data):
    '''Callback for the Intel Realsense D435i and OpenCV charuco pose estimation'''
    global pose
    pose = numpy.array([data.tx, data.ty, data.tz, data.rx, data.ry, data.rz])
    
def observation_callback(data):
    '''Callback for the entire observation array (same as in simulation)'''
    global observations
    observations = numpy.array(data.data)

In [8]:
# ROS control commands
def control(q):
    '''Controlling the Universal Robot UR10 JointGroupPositionController'''
    global ctrl_q
    command = Float64MultiArray()
    command.data = q
    control_publisher.publish(command)
    ctrl_q = q.copy()
    
def ft_zero():
    '''Zeroes the Robotiq FT300 force torque sensor by sending an ur_script via UR Secondary Interface'''
    zero_sensor_urscript = 'sec ft_set_zero():' + '\n' + '\t' + 'if(socket_open("127.0.0.1", 63350, "acc")):' \
          + '\n' + '\t' + '\t' + 'socket_send_string("SET ZRO", "acc")' \
         + '\n' + '\t' + '\t' + 'socket_close("acc")' + '\n' + '\t' + 'end' + '\n' + 'end' + '\n'
    ft_zero_publisher.publish(zero_sensor_urscript)

# Creation

Simulation Environment (Gym)

In [ ]:
model = mujoco_py.load_model_from_path(path)
sim = mujoco_py.MjSim(model)
q_init = numpy.array([0, -1.3, 2.1, -0.80, 1.5708, 0.0])

ctrl_q = q_init
set_state(q_init)

viewer = mujoco_py.MjViewer(sim)
viewer.render()

In [ ]:
class RandomizedEnv(ur10_noisy_pd_env.Ur10Env):
    def __init__(self, env_config):
        model_xml_path = os.path.join(*[HOME_PATH, "DRL_SetBot-RearVentilation", "UR10", "ur10_assembly_setup_rand_temp_{}.xml".format(env_config.worker_index)])
        randomize.randomize_ur10_xml(worker_id = env_config.worker_index)
        ur10_noisy_pd_env.Ur10Env.__init__(
            self, model_xml_path, n_substeps=80, distance_threshold=0.002, initial_qpos=initial_qpos_simpheg_conf2,
            reward_type='dense', ctrl_type='cartesian', vary=True, worker_id=env_config.worker_index,
            randomize_kwargs=randomize_kwargs)
        #utils.EzPickle.__init__(self)


register_env("RandomizedEnv", lambda config: RandomizedEnv(config))

Agent

In [ ]:
ray.init()
cls = get_trainable_cls("PPO")

In [ ]:
#agent = cls(env="RandomizedEnv", config=config)
agent = cls(config=config)
agent.restore(checkpoint)

In [ ]:
print(ray)
print(get_trainable_cls)
print(ray.tune.registry)
print(python)
clss = get_trainable_cls("PPO")

ROS-nodes

In [ ]:
rospy.init_node("drl_experiment", anonymous=True)
rospy.Subscriber("/joint_states", JointState, q_callback)
rospy.Subscriber("/ft300_force_torque", WrenchStamped, ft_callback)
rospy.Subscriber("/pose_estimation", EstimatedPose, pose_callback)
rospy.Subscriber("/observation", CustomVector, observation_callback)
control_publisher = rospy.Publisher("/joint_group_position_controller/command", Float64MultiArray, queue_size=1)
ft_zero_publisher = rospy.Publisher("/ur_hardware_interface/script_command", String, queue_size=1)

rate = rospy.Rate(125)
rate25 = rospy.Rate(25)

Data structures for storing and saving data

In [ ]:
observations_rec = []
poses_rec = []
ft_values_rec = []
q_rec = []
qd_rec = []
qdd_rec = []
ctrl_qs = []

# Run On Hardware

In [ ]:
agent.compute_action([0,0,0,0,0,0,0,0,0,0,0,0])

In [35]:
ctrl_q = actual_q
go_smooth(q_init) #go slowly to initial position
print(actual_q)

[ 1.19842252e-05 -1.29977495e+00  2.10471249e+00 -7.99800698e-01
  1.57093012e+00  5.38915629e-04]


In [ ]:
done = False
max_limit = 0.0001
ft_zero()

while not done:
    obs = get_obs()
    action = agent.compute_action(obs)
    for i in range(1):
        dx = max_limit * action.copy()
        rot_mat = sim.data.get_body_xmat('gripper_dummy_heg')
        dx_ = numpy.concatenate([rot_mat.dot(dx[:3]), rot_mat.dot(dx[3:])])  ## transform to right coordinate system
        dq  = get_dq(dx_)
        q = ctrl_q + dq
        control(q)
        
        observation_rec.append(observations)
        poses_rec.append(pose)
        ft_values_rec.append(ft_values)
        q_rec.append(actual_q)
        qd_rec.append(actual_qd)
        qdd_rec.append(actual_qdd)
        ctrl_qs.append(ctrl_q)
        
        #rate.sleep()
    rate.sleep()

# !!! Save recorded Data !!!

In [ ]:
run_number = 0
info = "basic"
date = (datetime.datetime.now()).strftime("%y%m%d")

observation_rec_df = pandas.DataFrame(observation_rec)
poses_rec_df = pandas.DataFrame(poses_rec)
ft_values_rec_df = pandas.DataFrame(ft_values_rec)
q_rec_df = pandas.DataFrame(q_rec)
qd_rec_df = pandas.DataFrame(qd_rec)
qdd_rec_df = pandas.DataFrame(qdd_rec)
ctrl_qs_df = pandas.DataFrame(ctrl_qs_df)

if not os.path.exists(os.path.join(*[SAVE_PATH, date])):
    os.makedirs(os.path.join(*[SAVE_PATH, date]))

observation_rec_df.to_feather(os.path.join(*[DIRECTORY, "{}_obs_{}_run{}.ftr".format(date, info, run)]))
poses_rec_df.to_feather(os.path.join(*[DIRECTORY, "{}_poses_{}_run{}.ftr".format(date, info, run)]))
ft_values_rec_df.to_feather(os.path.join(*[DIRECTORY, "{}_fts_{}_run{}.ftr".format(date, info, run)]))
q_rec_df.to_feather(os.path.join(*[DIRECTORY, "{}_qs_{}_run{}.ftr".format(date, info, run)]))
qd_rec_df.to_feather(os.path.join(*[DIRECTORY, "{}_qds_{}_run{}.ftr".format(date, info, run)]))
qdd_rec_df.to_feather(os.path.join(*[DIRECTORY, "{}_qdds_{}_run{}.ftr".format(date, info, run)]))
ctrl_qs_df.to_feather(os.path.join(*[DIRECTORY, "{}_ctrlqs_{}_run{}.ftr".format(date, info, run)]))

# --- Visualize the Data with visualize.ipynb ---